Application using altimetry data obtained from AVISO.
* Region of interest: Mediterranean Sea.
* Period of interest: May 2014.
* Satellite avaible: AltiKa, Cryosat, H2 and JASON-2. 

In [16]:
using divand
using NetCDF
using PyPlot
using Logging
Logging.configure(level=DEBUG);

# User inputs

## Domain of interest, resolution

In [33]:
lonmin, lonmax, latmin, latmax = -7., 40., 30., 48.
dx, dy, dt = .5, .25, 1.
lonr = lonmin:dx:lonmax;
latr = latmin:dy:latmax;
timer = 23510:dt:23529;

## File and directories

In [9]:
datadir = joinpath("../data/");
datafile = joinpath(datadir, "ADT_20140515_20140604.dat")
resultdir = joinpath("../results/divand/");
resultfile = joinpath(resultdir, "ADT_20140515_20140604.nc");
info("Data file:\n", datafile)
info("Result file:\n", resultfile)

INFO: Data file:
../data/ADT_20140515_20140604.dat
INFO: Result file:
../results/divand/ADT_20140515_20140604.nc


# Read data file (4-column, ascii)

In [10]:
function readAltimetryFile(fname)

    data = readlines(open(fname,"r"))
    nobs = length(data)

    lon = zeros(nobs)
    lat = zeros(nobs)
    time = zeros(nobs)
    field = zeros(nobs)

    for i in 1:nobs
        rec = split(data[i])
        lon[i] = parse(Float64,rec[1])
        lat[i] = parse(Float64,rec[2])
        time[i] = parse(Float64,rec[3])
        field[i] = parse(Float64,rec[4])
    end
    
    return lon, lat, time, field
end
info("Data read from file ", datafile)

INFO: Data read from file ../data/ADT_20140515_20140604.dat


In [11]:
lon, lat, time, field = readAltimetryFile(datafile);
info("Reading ", length(lon), " data points")
info("Mean field value = ", mean(field))

INFO: Reading 11371 data points
INFO: Mean field value = -0.09550065957259697


Another way to read the data (similar to python `append` method).<br>
In progress...

In [ ]:
npoints = 100 #should find a way to count line number without reading the whole file

open(datafile, "r") do f
    global lon, lat, ttime, field
    for line in eachline(f)
        l = split(line);
        lon = parse(Float64, l[1]);
        lat = parse(Float64, l[2]);
        ttime = parse(Float64, l[3]);
        field = parse(Float64, l[4]);
    end;       
end;

Create mask and metrics using coordinate vectors

In [34]:
mask, (px, py, pt), (xi, yi, ti) = divand_rectdom(lonr, latr, timer);

## Analysis parameters

In [35]:
# correlation length
len = (1., 1., 4.);
# obs. error variance normalized by the background error variance
epsilon2 = 1;

# Perform interpolation

In [36]:
@time fi,s = divandrun(mask, (px, py, pt), (xi, yi, ti), (lon, lat, time), field, len, epsilon2; alphabc=2);

 74.552338 seconds (7.51 M allocations: 9.136 GB, 3.28% gc time)


## Export results to netCDF

In [25]:
sz = size(mask)
dims = [NcDim("longitude",sz[1]), NcDim("latitude",sz[2]), NcDim("time",sz[3])];
nc = NetCDF.create(resultfile, NcVar("ADT", dims))
nc["ADT"][:,:, :] = fi
NetCDF.close(nc);
info("Written interpolated field in file:\n", resultfile)

INFO: Written interpolated field in file:
/home/ctroupin/Projects/201704_DivaAltimetry/results/divand/ADT_20140515_20140604.nc


# Plotting

Done in [another notebook in Python](../python/plot_AVISO_divand.ipynb) due to a problem with Julia / matplotlib.